# Psychic
本 Notebook 涵盖了如何从 `Psychic` 加载文档。更多详情请参阅 [此处](/docs/integrations/providers/psychic)。

## 先决条件
1. 遵循 [本文档](/docs/integrations/providers/psychic) 中的快速入门部分
2. 登录 [Psychic dashboard](https://dashboard.psychic.dev/) 并获取你的 secret key
3. 将前端 react 库安装到你的 Web 应用中，并让用户验证连接。连接将使用你指定的 connection id 来创建。

## 加载文档

使用 `PsychicLoader` 类从连接中加载文档。每个连接都有一个连接器 ID（对应已连接的 SaaS 应用程序）和一个连接 ID（您传递给前端库的 ID）。

In [7]:
# Uncomment this to install psychicapi if you don't already have it installed
!poetry run pip -q install psychicapi langchain-chroma


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [ ]:
from langchain_community.document_loaders import PsychicLoader
from psychicapi import ConnectorId

# Create a document loader for google drive. We can also load from other connectors by setting the connector_id to the appropriate value e.g. ConnectorId.notion.value
# This loader uses our test credentials
google_drive_loader = PsychicLoader(
    api_key="7ddb61c1-8b6a-4d31-a58e-30d1c9ea480e",
    connector_id=ConnectorId.gdrive.value,
    connection_id="google-test",
)

documents = google_drive_loader.load()

## 将文档转换为嵌入

我们现在可以将这些文档转换为嵌入，并将其存储在像 Chroma 这样的向量数据库中。

In [ ]:
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_chroma import Chroma
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings)
chain = RetrievalQAWithSourcesChain.from_chain_type(
    OpenAI(temperature=0), chain_type="stuff", retriever=docsearch.as_retriever()
)
chain({"question": "what is psychic?"}, return_only_outputs=True)